#  Capstone Project - The Battle of Neighborhoods (Week 1)

 ## Table of contents
* [Introduction](#introduction)
* [Data Acquisition](#data)

## Introduction <a name ="introduction"></a>

**LIDL** is one of the cheapest super market in Europe but it's not represented in many European cities, specially in Scandinavian countries such as Denmark. Surprisingly, the last five years, ** LIDL Group ** have started to build several ** LIDL Supermarket** in different regions of Denmark and specially the north of Denmark called ** Nordjylland ** in Danish language. Therefore, the purpose of this capstone project is to investigate and find out where the next ** LIDL Supermarket** will be built in **Aalborg** the main city of **Nordjylland**.
The target audience is Aalborg Municipality or borough and the stakeholders are of course ** LIDL Group **, the main competitors such as **Rema1000**, **Føtex**, **Fakta**, **ALDI** and ** Aalborg City**


## Data

To Tackle or solve this problem, I will of course nedd a Dataset, which will  find and locate all the existing  **Lidl** Supermarket and main competitors which already exist in Nordjylland region. I do not have an existing dataset for this problem. For that reason, **Foursquare API ** will be used in order to create a proper dataset that will be used to solve the problem. The dataset will certainly contain:
* All main competiors such as **Rema1000**, **Føtex**, **Fakta**, **ALDI** , ** Meny ** and their location
* All the existing **LIDl** supermarket,their geograpghic location and the ditance from  the main city.
* Aalborg borough or neighbourhood, the population and the number of person per  km2.